# Kaggle Competition: Predicting Red Hat Business Value
## Goal: Classify customer potential
Like most companies, Red Hat is able to gather a great deal of information over time about the behavior of individuals who interact with them. They’re in search of better methods of using this behavioral data to predict which individuals they should approach—and even when and how to approach them.

In this competition, Kagglers are challenged to create a classification algorithm that accurately identifies which customers have the most potential business value for Red Hat based on their characteristics and activities.

With an improved prediction model in place, Red Hat will be able to more efficiently prioritize resources to generate more business and better serve their customers.

The challenge of this competition is to predict the potential business value of a person who has performed a specific activity. The business value outcome is defined by a yes/no field attached to each unique activity in the activity file. The outcome field indicates whether or not each person has completed the outcome within a fixed window of time after each unique activity was performed.

## Data
This competition uses two separate data files that may be joined together to create a single, unified data table: a people file and an activity file.

The people file contains all of the unique people (and the corresponding characteristics) that have performed activities over time. Each row in the people file represents a unique person. Each person has a unique people_id.

The activity file contains all of the unique activities (and the corresponding activity characteristics) that each person has performed over time. Each row in the activity file represents a unique activity performed by a person on a certain date. Each activity has a unique activity_id.

The activity file contains several different categories of activities. Type 1 activities are different from type 2-7 activities because there are more known characteristics associated with type 1 activities (nine in total) than type 2-7 activities (which have only one associated characteristic).

To develop a predictive model with this data, you will likely need to join the files together into a single data set. The two files can be joined together using person_id as the common key. All variables are categorical, with the exception of 'char_38' in the people file, which is a continuous numerical variable.

## Evaluation Criteria
For each activity_id in the test set, you must predict a probability for the 'outcome' variable, represented by a number between 0 and 1
Evaluated on area under the ROC curve between the predicted and the observed outcome

### Reference
https://www.kaggle.com/c/predicting-red-hat-business-value/data

In [1]:
import os
import pandas as pd
import numpy as np

In [3]:
train_data = pd.read_csv('/Users/Rej1992/Documents/kaggle/Data/predicting-red-hat-business-value/act_train.csv')
test_data = pd.read_csv('/Users/Rej1992/Documents/kaggle/Data/predicting-red-hat-business-value/act_test.csv')
people = pd.read_csv('/Users/Rej1992/Documents/kaggle/Data/predicting-red-hat-business-value/people.csv')